# 4. Jobs Data: RAIS & Economic Geography

**The Problem:** RAIS data is strictly tabular. It tracks formal employment ties but doesn't map them.

**The Solution:** AtlasBR uses the `cep` (Postal Code) of the establishment to place it on the map. This allows us to visualize **Economic Density**.

**What you will learn:**
1.  **Mass Geocoding:** Mapping thousands of companies instantly.
2.  **Job Hubs:** Identifying where the jobs actually *are* versus where people live.

In [1]:
import sys
import os
from pathlib import Path

# --- DEVELOPER SETUP (Optional) ---
# If running locally without 'pip install', we add the '../src' folder to path.
current_path = Path(os.getcwd())
if current_path.name == "tutorials":
    # Go up one level to root, then into 'src' (if using src-layout) or just root (flat-layout)
    root_dir = current_path.parent
    src_dir = root_dir / "src"
    
    if src_dir.exists():
        sys.path.append(str(src_dir))
    else:
        sys.path.append(str(root_dir))

import atlasbr

In [2]:
from atlasbr.app.rais import load_rais

atlasbr.configure_logging()
MUNICIPALITY = "Belo Horizonte, MG"

## 1. Fetching the "Job Map"
We load RAIS establishments for 2021.
*Note: This dataset is large. The CEP join might take a few seconds.*

In [3]:
gdf_jobs = load_rais(
    places=[MUNICIPALITY],
    year=2021,
    geocode=True, # Essential for spatial analysis
    gcp_billing=os.getenv("GOOGLE_CLOUD_PROJECT")
)

print(f"💼 Loaded {len(gdf_jobs)} active establishments.")

2025-12-16 19:41:22,140 - atlasbr - INFO -     🌍 Fetching municipality metadata from geobr...
2025-12-16 19:41:22,851 - atlasbr - INFO -     ℹ️  Resolved 'Belo Horizonte, MG' -> 3106200
2025-12-16 19:41:22,858 - atlasbr - INFO -     🏭 Loading RAIS 2021 via strategy 'bd_table'...
2025-12-16 19:41:23,671 - atlasbr - INFO -     🏭 Fetching RAIS 2021 from Base dos Dados...


Downloading: 100%|██████████|


2025-12-16 19:41:40,344 - atlasbr - INFO -     📍 Fetching CEP coordinates from Base dos Dados...


Downloading: 100%|██████████|

2025-12-16 19:41:43,619 - atlasbr - INFO -     🌍 Geocoding 174097 establishments via CEP...


2025-12-16 19:41:46,715 - atlasbr - INFO - ✅ Loaded 174097 establishments (Geolocated).


💼 Loaded 174097 active establishments.


## 2. Who is hiring?
Using CNAE (Economic Activity) codes to see the city's profile.

In [4]:
# Top 5 sectors by number of establishments
top_sectors = gdf_jobs["cnae_2"].value_counts().head(5)
print("Top Economic Sectors:")
print(top_sectors)

Top Economic Sectors:
cnae_2
81125    13423
86305     7229
56112     6867
71120     5356
41204     4144
Name: count, dtype: int64


## 3. Interactive Map: Employment Density
Instead of mapping every corner store, let's filter for **Large Employers** (> 50 employees).
This visualizes the Central Business District (CBD) and industrial zones.

In [8]:
large_employers = gdf_jobs[gdf_jobs["quantidade_vinculos_ativos"] > 500].copy()

print(f"Mapping {len(large_employers)} large employers...")

large_employers.explore(
    column="quantidade_vinculos_ativos",
    cmap="inferno_r", # Hot colors for high job counts
    marker_kwds={"radius": 8},
    scheme="quantiles",
    tooltip=[
        "cnae_2", 
        "quantidade_vinculos_ativos"
    ],
    tiles="CartoDB DarkMatter",
    legend_kwds={"caption": "Number of Active Jobs"}
)

Mapping 230 large employers...
